**Cryptocurrency Prediction using DNN Models**

Import the necessary libraries

In [0]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from keras import Sequential
from keras.layers import Dense, LSTM, CuDNNLSTM, BatchNormalization, Bidirectional, Dropout, Activation

Load the data from the csv file onto the pandas dataframe.

In [0]:
def _get_data(filename):
  df = pd.read_csv(filename, names = ['time','low','high', 'open', 'close', 'volume'], dtype = float)
  fig = plt.figure(figsize=(10,8))
  df['close'].plot(label='bid')
  plt.grid()
  plt.legend()
  plt.show()
  
  return df

The data is normalized using a MinMaxScaler() from Sklearn package. The top 10k samples are taken for testing, as they are the most recent timestamps. The remaining dataset is taken for training.

In [0]:
def _data_preprocess(df):
  data = df.iloc[:,4:5].astype('float').values
  
  scaler = MinMaxScaler()
  data = scaler.fit_transform(data)
  
  training_set = data[10001:len(data)]
  testing_set = data[10000:]
  
  X_train = training_set[0:len(training_set)-1]
  Y_train = training_set[1:len(training_set)]
  
  X_test = testing_set[0:len(testing_set)-1]
  Y_test = testing_set[1:len(testing_set)]
  
  X_train = np.reshape(X_train, (len(X_train), 1, X_train.shape[1]))
  X_test = np.reshape(X_test, (len(X_test),1,X_test.shape[1]))
  
  return X_train, Y_train, X_test, Y_test, scaler

Model1 :  This is a simple 3layer DNN model. It has two layers of LSTM and one dense layer with softmax activation function. The model is compiled with adam optimizer, mean squared error loss function.

In [0]:
def _model1(X_train,Y_train):
  model = Sequential()
  model.add(LSTM(256, return_sequences = True, input_shape=(X_train.shape[1], X_train.shape[2])))

  model.add(LSTM(256))
  model.add(Dense(1))
  
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(X_train, Y_train, epochs=30, batch_size=64, validation_split=0.1, shuffle=False)
  
  return model

Model2 : This is the initial base model. This was just used for comparison purposes. This has three sets of LSTM layers - with each set having one LSTM layer, a dropout layer and a batch normalization. There are two dense layers in the end. 

In [0]:
def _model2(x_train, y_train):
  model = Sequential()
  model.add(LSTM(128, input_shape=(x_train.shape[1], X_train.shape[2]), return_sequences=True))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(LSTM(128, return_sequences=True))
  model.add(Dropout(0.1))
  model.add(BatchNormalization())

  model.add(LSTM(128))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.2))

  model.add(Dense(1, activation='softmax'))
  
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(x_train, y_train, epochs=10, batch_size=64, shuffle=False)
  
  return model

Model3: Bidirectional LSTM is used here. There are three sets of Bidirectional LSTM, each with a dropout layer. Here linear activation function is used. Finally there is a dense layer. The model is compiled with adam optimizer and mean squared error loss function. 

In [0]:
def _model3(X_train, Y_train, window_size=32, dropout_value=0.2,activation_function='linear'):
  model = Sequential()
  model.add(Bidirectional(LSTM(window_size, return_sequences=True), input_shape=(X_train.shape[1],X_train.shape[2]),))
  model.add(Dropout(dropout_value))
  model.add(Bidirectional(LSTM((window_size*2), return_sequences=True)))
  model.add(Dropout(dropout_value))
  model.add(Bidirectional(LSTM(window_size, return_sequences=False)))
  model.add(Dense(units=1))   
  model.add(Activation(activation_function))
  model.compile(loss='mean_squared_error', optimizer='adam')
  model.fit(X_train, Y_train, epochs=10, batch_size=64, shuffle=False)
    
  return model

The predicted prices and the real prices are plotted against each other, to visualize the model performance.

In [0]:
def _plot_predictions(model, X_test, Y_test, scaler, fname):
  predicted_prices = model.predict(X_test)
  predicted_prices = scaler.inverse_transform(predicted_prices)
  real_price = scaler.inverse_transform(Y_test)
  
  plt.figure(figsize=(20,8))
  plt.plot(predicted_prices, color='red',label='Predicted prices')
  plt.plot(real_price, color='blue', label='Real prices')
  plt.title('Predicted vs real prices')
  plt.xlabel('Time')
  plt.ylabel('Price')
  plt.legend(['Predicted price', 'Actual price'], loc='upper left')
  #plt.show()
  plt.savefig(fname+'-result')

The entire routine is called for each of the four cryptocurrencies.

In [0]:
def call_procedure(filename, fname):
  df = _get_data(filename)
  X_train, Y_train, X_test, Y_test, scaler = _data_preprocess(df)
  model = _model3(X_train,Y_train)
  print(model.summary())
  _plot_predictions(model, X_test, Y_test, scaler, fname)

In [0]:
def main():
  main_df = pd.DataFrame()
  ratios = ["BTC-USD", "BTC-USD", "ETH-USD", "LTC-USD"]
  for ratio in ratios:
    print(ratio)
    dataset = f'{ratio}.csv'
    call_procedure(dataset, ratio)

In [0]:
if __name__ == '__main__':
  main()